Checks if Panda jobs and taks data are indexed
====
This notebook checks number of indexed documents in jobs and panda tables and alerts if any of them is 0. It sends mails to all the people substribed to that alert. It is run every 6h from a cron job.

In [1]:
import subscribers, alerts
from datetime import datetime
from elasticsearch import Elasticsearch, exceptions as es_exceptions
es = Elasticsearch(hosts=[{'host':'cl-analytics.mwt2.org', 'port':9200}],timeout=60)

In [2]:
ct  = datetime.now()
ind = 'jobs_archive_%d-%02d-%02d' % (ct.year, ct.month, ct.day)
print(ind)
currentTime=int(round(datetime.now().timestamp()* 1000))
lastHours=7
startTime=currentTime - lastHours*3600000
print('start time', startTime)
print('current time',datetime.now())

jobs_archive_2017-01-07
start time 1483786083053
current time 2017-01-07 11:48:03.053649


In [3]:
jobs_query={
   "size": 0,
   "query": {
     "range":{"modificationtime":{"gte": startTime,"lte": currentTime}}
   }
}

res = es.search(index=ind, body=jobs_query, request_timeout=120)
print(res)

{'took': 3, 'hits': {'total': 292193, 'max_score': 0.0, 'hits': []}, 'timed_out': False, '_shards': {'total': 5, 'failed': 0, 'successful': 5}}


In [4]:
if res['hits']['total'] == 0:
    S = subscribers.subscribers()
    A = alerts.alerts()

    testName = 'Panda jobs'
    subscribersToJobsImporter =  S.getSubscribers(testName)
    print (subscribersToJobsImporter)
    for subscriber in subscribersToJobsImporter:
        body = 'Dear ' + subscriber[0]+',\n\n'
        body = body + '\tthis mail is to let you that there is an issue in indexing Panda Jobs data in UC Elasticsearch.\n'
        body = body + '\nBest regards,\nATLAS AAS'
        body = body + '\n\n To change your alerts preferences please you the following link:\n'+subscriber[2]
        print(subscriber)
        A.sendMail(testName, subscriber[1], body)
        A.addAlert(testName, subscriber[0],str(res))

In [5]:
ind = 'tasks_archive_%d-%02d' % (ct.year, ct.month)
tasks_query={
   "size": 0,
   "query": {
     "range":{"modificationtime":{"gte": startTime,"lte": currentTime}}
   }
}

res = es.search(index=ind, body=tasks_query, request_timeout=120)
print(res)

{'took': 1, 'hits': {'total': 2595, 'max_score': 0.0, 'hits': []}, 'timed_out': False, '_shards': {'total': 5, 'failed': 0, 'successful': 5}}


In [6]:
if res['hits']['total'] == 0:
    S = subscribers.subscribers()
    A = alerts.alerts()

    testName = 'Jedi tasks'
    subscribersToTasksImporter =  S.getSubscribers(testName)
    for subscriber in subscribersToTasksImporter:
        body = 'Dear ' + subscriber[0]+',\n\n'
        body = body + '\tthis mail is to let you that there is an issue in indexing Jedi tasks data in UC Elasticsearch.\n'
        body = body + '\nBest regards,\nATLAS AAS'
        body = body + '\n\n To change your alerts preferences please you the following link:\n'+subscriber[2]
        print(subscriber)
        A.sendMail(testName, subscriber[1], body)
        A.addAlert(testName, subscriber[0],str(res))